In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam

import seaborn as sns
sns.set()

Using TensorFlow backend.


# Data 

In [2]:
data = pd.read_csv('D:/IITM/DeepCache/syntheticDataset_O50.csv')
data.head()

,object_ID,request_time
0,5,2.961472
1,25,3.274127
2,2,3.785475
3,2,4.455687
4,4,5.288994


In [3]:
data = np.array(data)
req = data[:,0]

len(req)

292146

In [4]:
# to find the probability of occurance of Oi object is caculated as Ni/1000
# where Ni is number of occurance of Oi in past 1k req

count = np.zeros(50)
for i in range(1000):
    for j in range(50):
        if req[i] == j+1:
            count[j] += 1

prob = []            
for i in range(1000,len(req)):
    t = int(req[i] - 1)
    count[t] += 1
    t = int(req[i-1000] - 1)
    count[t] -= 1
    prob.append(count/1000)

prob = np.array(prob)
prob.shape

(291146, 50)

In [5]:
encoder_input_data = prob[:250000,:].reshape(20,-1,50) # total samples = 20
batch_size = 12500
decoder_target_data = []
for i in range(1,21):
    temp = []
    for j in range(10): # to predict next 10 req for every batch
        temp.append(prob[(batch_size*i) + j,:])
    decoder_target_data.append(temp)    

decoder_target_data = np.array(decoder_target_data).reshape(20,10,-1)

print(encoder_input_data.shape,decoder_target_data.shape)

(20, 12500, 50) (20, 10, 50)


In [6]:
decoder_input_data = np.zeros(decoder_target_data.shape)
decoder_input_data[1:,:,:] = decoder_target_data[:-1,:,:]
decoder_input_data[0,:,:] = encoder_input_data[-1,-1,:]
decoder_input_data.shape

(20, 10, 50)

# Model

In [7]:
latent_dim = 128 # LSTM hidden units
dropout = 0

In [8]:
encoder_inputs = Input(shape=(None, 50)) 
encoder = LSTM(latent_dim, dropout=dropout, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [9]:
decoder_inputs = Input(shape=(None, 50)) 
decoder_lstm = LSTM(latent_dim, dropout=dropout, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)


In [10]:
#decoder_dense = Dense(decoder_outputs) # 1 continuous output at each timestep
decoder_outputs = Dense(50, activation='softmax')(decoder_outputs)


In [11]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [12]:
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 50)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 50)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 91648       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 128),  91648       input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

# Compile The Model

In [13]:
lossNames = ["loss",]


model.compile(Adam(), loss='mean_squared_error')
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=30)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
20/20 [==============================] - 113s 6s/step - loss: 9.4625e-04
Epoch 2/30
20/20 [==============================] - 112s 6s/step - loss: 9.4552e-04
Epoch 3/30
20/20 [==============================] - 104s 5s/step - loss: 9.4469e-04
Epoch 4/30
20/20 [==============================] - 85s 4s/step - loss: 9.4379e-04
Epoch 5/30
20/20 [==============================] - 96s 5s/step - loss: 9.4284e-04
Epoch 6/30
20/20 [==============================] - 89s 4s/step - loss: 9.4185e-04
Epoch 7/30
20/20 [==============================] - 81s 4s/step - loss: 9.4083e-04
Epoch 8/30
20/20 [==============================] - 79s 4s/step - loss: 9.3977e-04
Epoch 9/30
20/20 [==============================] - 83s 4s/step - loss: 9.3869e-04
Epoch 10/30
20/20 [==============================] - 84s 4s/step - loss: 9.3757e-04
Epoch 11/30
20/20 [==============================] - 87s 4s/step - lo

# Testing The Model 

In [19]:
encoder_model = Model(encoder_inputs, encoder_states)
pred_steps = 10

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_outputs = Dense(50, activation='softmax')(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [20]:
def decode_sequence(input_seq):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, 50))
    
    # Populate the first target sequence with end of encoding series pageviews
    target_seq[0, 0, :] = input_seq[0, -1, :]

    # Sampling loop for a batch of sequences - we will fill decoded_seq with predictions
    # (to simplify, here we assume a batch of size 1).

    decoded_seq = np.zeros((1,pred_steps,50))
    for i in range(pred_steps):
        
        output, h, c = decoder_model.predict([target_seq] + states_value)
        
        decoded_seq[0,i,:] = output[0,0,:]

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, 50))
        target_seq[0, 0, :] = output[0,0,:]

        # Update states
        states_value = [h, c]

    return decoded_seq

In [21]:
sample = prob[250000:275000].reshape(1,25000,50)


In [24]:
prediction = decode_sequence(sample)
prediction.shape

(1, 10, 50)